In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn import metrics
import time, datetime
import itertools as it
from IPython.display import display
import pickle

In [27]:
from sklearn.model_selection import KFold
import torch
import sklearn.preprocessing 
from sklearn.decomposition import PCA

In [2]:
train_path="../../DATA/SWaT/SWaT_Physical/SWaT_Dataset_Normal_v0.csv"
test_path="../../DATA/SWaT/SWaT_Physical/SWaT_Dataset_Attack_v0.csv"

In [3]:
normal_data = pd.read_csv(train_path, index_col = 0)
attack_data = pd.read_csv(test_path, index_col = 0)

x_col = list(normal_data.columns)[:-1]
y_col = list(normal_data.columns)[-1]

X_normal = normal_data[x_col]
Y_normal = normal_data[y_col]

X_attack = attack_data[x_col]
Y_attack = attack_data[y_col]



In [16]:
cv = KFold(5, shuffle=False, random_state=0)

cv_list = []

for i, (idx_train, idx_test) in enumerate(cv.split(attack_data)):
    cv_list.append([idx_train, idx_test])


[ 89984  89985  89986 ... 449916 449917 449918]
[     0      1      2 ... 449916 449917 449918]
[     0      1      2 ... 449916 449917 449918]
[     0      1      2 ... 449916 449917 449918]
[     0      1      2 ... 359933 359934 359935]


In [ ]:
X_normal, Y_normal, X_attack, Y_attack, cv_list

In [33]:
def pca_return(x_train, x_test):
    scaler = sklearn.preprocessing.StandardScaler(with_std = False)
    scaler.fit(x_train)
    
    # train zero center about train data. For fairness test dataset scaled from x_train information
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    pca = PCA(n_components = np.shape(x_train)[1])
    pca.fit(x_train)
    
    x_train = pca.transform(x_train)
    x_test = pca.transform(x_test)
    
    return x_train, x_test


def lstm_dataset(train_path, test_path):
    x_train, y_train, x_test, y_test = base_dataset(train_path, test_path)

    # SInce sklearn osvm property, set atk = -1, normal = 1
    y_train[y_train == 'Attack'] = 1
    y_train[y_train == 'Normal'] = 0
    y_test[y_test == 'Attack'] = 1
    y_test[y_test == 'Normal'] = 0
    
    y_train = torch.FloatTensor(y_train).cuda()
    y_test = torch.FloatTensor(y_test).cuda()    
    
    x_train, x_test = pca_return(x_train, x_test)
    
    x_train = torch.FloatTensor(x_train).cuda()
    x_test = torch.FloatTensor(x_test).cuda()
    
    return  x_train, x_test,  y_test

def base_dataset(train_path, test_path, isprint = False):
    #load data
    normal_data = pd.read_csv(train_path, index_col = 0)
    attack_data = pd.read_csv(test_path, index_col = 0)

    x_col = list(normal_data.columns)[:-1]
    y_col = list(normal_data.columns)[-1]

    X_normal = normal_data[x_col]
    Y_normal = normal_data[y_col]

    X_attack = attack_data[x_col]
    Y_attack = attack_data[y_col]
    
    if (isprint == True):
        unique, counts = np.unique(Y_normal, return_counts=True)
        print('In train :', dict(zip(unique, counts)))
        unique, counts = np.unique(Y_attack, return_counts=True)
        print('In test :', dict(zip(unique, counts)))     
        print("data length is {}".format(len(x_col)))
    
    return X_normal, Y_normal, X_attack, Y_attack

def svm_dataset(train_path, test_path):
    x_train, y_train, x_test, y_test = base_dataset(train_path, test_path)

    # SInce sklearn osvm property, set atk = -1, normal = 1
    y_train[y_train == 'Attack'] = -1
    y_train[y_train == 'Normal'] = +1
    y_test[y_test == 'Attack'] = -1
    y_test[y_test == 'Normal'] = +1      

    x_train, x_test = pca_return(x_train, x_test)

    x_train = pd.DataFrame(data = x_train
             , columns = ['PC%d'%(i) for i in range(np.shape(x_train)[1])])

    x_test = pd.DataFrame(data = x_test
             , columns = ['PC%d'%(i) for i in range(np.shape(x_train)[1])])

    # Moving average Window
    x_train = x_train.rolling(10, min_periods = 1).mean()
    x_test = x_test.rolling(10, min_periods = 1).mean()

    y_train = y_train.rolling(10, min_periods=1).apply(check)
    y_test = y_test.rolling(10, min_periods=1).apply(check)


    return  x_train.values, x_test.values,  y_test.values

In [28]:
x_train, x_test,  y_test    = lstm_dataset(train_path, test_path)

In [31]:
x_test[cv_list[0][0]].shape

torch.Size([359935, 51])

In [32]:
x_test[cv_list[0][1]].shape

torch.Size([89984, 51])

In [35]:
def check(data) :
    if -1 in data :
        return -1
    else : 
        return 1

In [36]:
x_train, x_test,  y_test    = svm_dataset(train_path, test_path)

/home/mjh319/anaconda3/envs/latest_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:81: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
/home/mjh319/anaconda3/envs/latest_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:82: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning


In [37]:
x_test[cv_list[0][0]].shape

(359935, 51)

In [38]:
x_test[cv_list[0][1]].shape

(89984, 51)

In [2]:
y_true = np.array([0, 0, 1, 1])
y_scores = np.array([0.1, 0.4, 0.35, 0.8])
precision, recall, thresholds = precision_recall_curve(
  y_true, y_scores)

NameError: name 'precision_recall_curve' is not defined

In [1]:
 precision  

 recall

thresholds

IndentationError: unexpected indent (<ipython-input-1-531f97ff8a9b>, line 5)